In [1]:
import sys
fileDir = "/home/jovyan/notebooks"
sys.path.append(fileDir)

from utilities import *
from github import Github
import json
from github import Auth
import pandas as pd
import time

In [2]:
session = SparkSession.builder \
    .appName("Data integration") \
    .master(SPARK_MASTER_URL) \
    .config("spark.executor.memory", "5G") \
    .config("spark.authenticate", "false") \
    .getOrCreate()

In [3]:
HDFS_URL = "hdfs://namenode:9000//data-team"
PREFIX = "sample_"  # "sample_" or ""

### Reading from HDFS

In [4]:
# cambia se ti serve il dataset completo
repositories = session.read.json(f"{HDFS_URL}/{PREFIX}repositories.json")
repositories_names = repositories.select("repo_name").rdd.flatMap(lambda x: x).collect()

### Retrieve additional data from the GitHub API and dump it to a CSV

In [5]:
# Import github token from json file
credentials = json.load(open(f'{fileDir}/credentials.json'))

# using an access token
auth = Auth.Token(credentials['token'])

# Public Web Github
g = Github(auth=auth)

In [7]:
features = ["forks_count", "default_branch",
            "open_issues_count", "created_at","stargazers_count",
            "language", "topics", "visibility"]
columns_names = ["repo_name"] + features

repo_data_df = pd.DataFrame(columns=columns_names)

for repo_name in repositories_names:
    try:
        attributes_map = collectAttributes(g.get_repo(repo_name), features)
    except:
        print("Error in collecting attributes for repo: " + repo_name)
        continue
    attributes = [attributes_map[f] for f in features]

    time.sleep(1.5) # avoid to exceed the rate limit of GitHub API

    new_row = [repo_name] + attributes

    repo_data_df.loc[len(repo_data_df)] = new_row

Following Github server redirection from /repos/firehol/netdata to /repositories/10744183
Following Github server redirection from /repos/ParsePlatform/parse-server to /repositories/50603846
Following Github server redirection from /repos/vhf/free-programming-books to /repositories/13491895
Following Github server redirection from /repos/caesar0301/awesome-public-datasets to /repositories/26898879
Following Github server redirection from /repos/airbnb/caravel to /repositories/39464018
Following Github server redirection from /repos/googlesamples/android-architecture to /repositories/51148780


Error in collecting attributes for repo: Rochester-NRT/AlphaGo


Following Github server redirection from /repos/robbyrussell/oh-my-zsh to /repositories/291137
Following Github server redirection from /repos/facebook/draft-js to /repositories/52113921
Following Github server redirection from /repos/matryer/bitbar to /repositories/14376285
Following Github server redirection from /repos/ptmt/react-native-desktop to /repositories/43640830
Following Github server redirection from /repos/parkjs814/AlgorithmVisualizer to /repositories/142092876
Following Github server redirection from /repos/hollance/swift-algorithm-club to /repositories/50447720
Following Github server redirection from /repos/jiahaog/nativefier to /repositories/38558578
Following Github server redirection from /repos/reactjs/redux to /repositories/36535156
Following Github server redirection from /repos/mbostock/d3 to /repositories/943149
Following Github server redirection from /repos/thejameskyle/the-super-tiny-compiler to /repositories/53639099
Following Github server redirection fro

Error in collecting attributes for repo: CRYTEK-CRYENGINE/CRYENGINE


Following Github server redirection from /repos/atom/electron to /repositories/9384267
Following Github server redirection from /repos/mxstbr/react-boilerplate to /repositories/30969188
Following Github server redirection from /repos/kadirahq/react-storybook to /repositories/54173593
Following Github server redirection from /repos/rg3/youtube-dl to /repositories/1039520
Following Github server redirection from /repos/daneden/animate.css to /repositories/2561582
Following Github server redirection from /repos/docker/docker to /repositories/7691631
Following Github server redirection from /repos/shockone/black-screen to /repositories/32665718
Following Github server redirection from /repos/google/xi-editor to /repositories/57167764
Following Github server redirection from /repos/weui/weui to /repositories/28167802
Following Github server redirection from /repos/open-source-society/computer-science to /repositories/19415064
Following Github server redirection from /repos/puikinsh/gentelel

...

Error in collecting attributes for repo: DreaminginCodeZH/Douya


Following Github server redirection from /repos/timothycrosley/hug to /repositories/39233228
Following Github server redirection from /repos/itsabot/abot to /repositories/51855345
Following Github server redirection from /repos/Homebrew/homebrew to /repositories/206084
Following Github server redirection from /repos/Microsoft/BotBuilder to /repositories/53092757
Following Github server redirection from /repos/legomushroom/mojs to /repositories/21234201
Following Github server redirection from /repos/Itseez/opencv to /repositories/5108051
Following Github server redirection from /repos/numbbbbb/the-swift-programming-language-in-chinese to /repositories/20429943
Following Github server redirection from /repos/mattermost/platform to /repositories/37448451


Error in collecting attributes for repo: wardowsmcc/badass


Following Github server redirection from /repos/realm/realm-cocoa to /repositories/4044891
Following Github server redirection from /repos/AngularClass/angular2-webpack-starter to /repositories/31829770
Following Github server redirection from /repos/gaearon/redux-devtools to /repositories/38565895
Following Github server redirection from /repos/mitsuhiko/flask to /repositories/596892
Following Github server redirection from /repos/collinhundley/Swift-AI to /repositories/46527265
Following Github server redirection from /repos/necolt/Swifton to /repositories/52214257
Following Github server redirection from /repos/facebook/flux to /repositories/22046023
Following Github server redirection from /repos/UnivaqTelegramBot/UnivaqInformaticaBot to /repositories/51418726
Following Github server redirection from /repos/equinusocio/material-theme to /repositories/36674234
Following Github server redirection from /repos/AutumnsWind/Front-end-tutorial to /repositories/42237651
Following Github se

Error in collecting attributes for repo: css-modules/css-modules
Error in collecting attributes for repo: spf13/spf13-vim
Error in collecting attributes for repo: facebook/flow
Error in collecting attributes for repo: ageitgey/amplify
Error in collecting attributes for repo: Netflix/Hystrix
Error in collecting attributes for repo: atConf/atswift-2016-resources
Error in collecting attributes for repo: zquestz/s
Error in collecting attributes for repo: LukyVj/family.scss
Error in collecting attributes for repo: redspread/spread
Error in collecting attributes for repo: overtrue/wechat
Error in collecting attributes for repo: reactjs/react-basic
Error in collecting attributes for repo: phanan/koel
Error in collecting attributes for repo: bboyfeiyu/android-tech-frontier
Error in collecting attributes for repo: eslint/eslint
Error in collecting attributes for repo: hongyangAndroid/okhttp-utils
Error in collecting attributes for repo: angular/angular-cli
Error in collecting attributes for rep

In [8]:
spark_df = session.createDataFrame(repo_data_df)

# save to csv format in the hdfs
spark_df.repartition(1) \
        .withColumn("topics", spark_df["topics"].cast("string")) \
        .write.csv(f"{HDFS_URL}/{PREFIX}repo_API_data.csv", header=True, mode="overwrite")

In [9]:
session.sparkContext.stop()
session.stop()